In [1]:
import pandas as pd
import sys
sys.path.insert(0,"./..")
from src.PassivePy import PassivePyAnalyzer
from src import analysis_passive

analyzer = PassivePyAnalyzer(spacy_model= "fr_core_news_lg") # (0.70, 0.34)
#analyzer = PassivePyAnalyzer(spacy_model= "fr_dep_news_trf") #(0.75, 0.37)


import spacy
from spacy.matcher import Matcher
#nlp = spacy.load("fr_core_news_lg")

In [22]:
annotation = pd.read_json("/home/robin/Data/Etude_1000/annotation/20221208_voix_passive.jsonl", lines =True)

In [23]:
def cut_text(line):
    labels = line["clean_label"]
    text = line['clean_text']
    i_cut = -1
    for label in labels : 
        if label[-1] == "Fin" :
            i_cut = label[1]
            break
    return text[:i_cut]

def clean_data(annotation):
    annotation['clean_text'] = annotation['text'].apply(lambda x : x.replace("  "," "))
    clean_annotation = []
    for i in range(len(annotation)):
        ex_label = annotation["label"].iloc[i]
        ex_text = annotation["text"].iloc[i]
        clean_ex_label = []
        for ann in ex_label :
            s = ann[0]
            e = ann[1]
            label = ann[2]
            diff_e = len(ex_text[:e]) - len(ex_text[:e].replace("  "," ")) # count how many caraters was deleted
            diff_s = len(ex_text[:s]) - len(ex_text[:s].replace("  "," "))
            clean_ex_label.append([s-diff_s,e-diff_e,label])
        clean_annotation.append(clean_ex_label)
    annotation['clean_label'] = clean_annotation

    return annotation

k=10
test  = clean_data(annotation.iloc[:k])
test['cut_text'] = test.apply(cut_text, axis=1)


/tmp/ipykernel_292388/4097232454.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation['clean_text'] = annotation['text'].apply(lambda x : x.replace("  "," "))
/tmp/ipykernel_292388/4097232454.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation['clean_label'] = clean_annotation
/tmp/ipykernel_292388/4097232454.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [ ]:
def check_if_found(x,test_label,seuil = 30) :
    s = x[0]
    e = x[1]
    for t_label in test_label :
        t_s = t_label[0]
        t_e = t_label[1]
        if abs( t_s-s)<seuil:
            return 1
        else :
            continue
    
    return 0

def check_if_found_2(s,e,test_label,seuil = 30) :
    for t_label in test_label :
        t_s = t_label[0]
        t_e = t_label[1]
        if abs( t_s-s)<seuil:
            return 1
        else :
            continue
    
    return 0

def find_ecart_minimum(x,all_start) :
    min = 200
    for elt in all_start:
        if elt != x :
            if abs(elt -x)< min :
                min = abs(elt -x)
    return min

def eval_doc(x, seuil = 35):
    example =  x['cut_text']
    label = x['clean_label']
    test_label = []
    matches, doc = analyzer.prepare_visualisation(example)
    for span in doc.spans["passive"] :
        span_text = doc[span.start-2: span.end+2].text
        test_label.append([span.start_char, span.end_char, span.label_, span_text])

    # removing the doublon caused by the diffferent rules
    df_test = pd.DataFrame(test_label, columns = ['start',"end","label","text"])
    all_start = df_test['start'].tolist()
    df_test['min_ecart'] = df_test['start'].apply(lambda x : find_ecart_minimum(x,all_start))
    n_test = len(df_test[df_test['min_ecart']>40])

    # find the one False positive
    df_test['FP_found'] = df_test.apply(lambda x: check_if_found_2(x.start,x.end,label,seuil),axis=1)
    FP_errors = df_test[df_test['FP_found']==0]
    # Counting the matching passive found (labelisation is not precise) 
    df_eval = pd.DataFrame()
    df_eval['true'] = label
    df_eval['ex'] = df_eval["true"].apply(lambda x : example[x[0]-2:x[1]])
    df_eval['is_found']  = df_eval['true'].apply(lambda x : check_if_found(x,test_label, seuil))

    n_VP = len(df_eval[df_eval['is_found']==1])
    FN_errors = df_eval[df_eval['is_found']==0]
    n_true = len(df_eval['is_found'])

    return n_VP, n_true, n_test, FN_errors, FP_errors

test['n_VP'], test['n_true'] , test['n_test'], test['FN_errors'], test['FP_errors']= zip(*test.apply(eval_doc, axis=1))
test['recall'] =  test['n_VP']/test['n_true']
test['precision'] =  test['n_VP']/test['n_test']
print("recall:", test['n_VP'].sum()/test['n_true'].sum())
print("precision:", test['n_VP'].sum()/test['n_test'].sum())

In [6]:
for i in range(8) :
    test['FP_errors'][i].sort_values("start").to_csv(f'/home/robin/Code_repo/app_passive_detection/error_analysis/{i}_FP_lg.csv')

In [6]:
RAPPEL = []
PREC = []
for seuil in [10,20,30,40,50]:
    test['n_VP'], test['n_true'] , test['n_test']= zip(*test.apply(lambda x : eval_doc(x,seuil), axis=1))
    rappel = test['n_VP'].sum()/test['n_true'].sum()
    precision = test['n_VP'].sum()/test['n_test'].sum()
    RAPPEL.append(rappel)
    PREC.append(precision)

In [7]:
print(RAPPEL)
print(PREC)

[0.4992867332382311, 0.6504992867332382, 0.6875891583452212, 0.7104136947218259, 0.7132667617689016]
[0.7972665148063781, 0.8803088803088803, 0.7626582278481012, 0.680327868852459, 0.6067961165048543]


In [14]:
example =  test['cut_text'][0]
test_label = []
matches, doc = analyzer.prepare_visualisation(example)
for span in doc.spans["passive"] :
    test_label.append([span.start_char, span.end_char, span.label_, span.text])

# removing the doublon because of diffferents rules
df_test = pd.DataFrame(test_label, columns = ['start',"end","label","text"])
all_start = df_test['start'].tolist()
df_test['min_ecart'] = df_test['start'].apply(lambda x : find_ecart_minimum(x,all_start))
n_test = len(df_test[df_test['min_ecart']<10])

In [100]:
nlp = spacy.load('fr_core_news_lg')
example =  test['cut_text'][0]
label = test['clean_label'][0]
doc = nlp.make_doc(example)
spans = []
for elt in label : 
    s = elt[0]
    e = elt[1]
    span = doc.char_span(s, e, label="passif", alignment_mode="contract")
    spans.append(span)
doc.spans["passive"] =  spans

In [101]:
true_label = []
for span in doc.spans["passive"] :
    true_label.append([span.start_char, span.end_char, span.label_,span.start, span.end, span.text])

In [104]:
df_true = pd.DataFrame(true_label, columns = ["char_start","char_end","label","start","end","text"])

In [113]:
import stanza
nlp_stanza = stanza.Pipeline("fr", disable = ["mwt","ents"], verbose =False)

In [ ]:
nlp

In [110]:
import spacy_stanza
nlp_stanza = spacy_stanza.load_pipeline("fr", disable = ["mwt","ents"], verbose =False)
analyzer = PassivePyAnalyzer(nlp = nlp_stanza)


In [ ]:
test_label = []
# compute the passive with automated method
matches, doc = analyzer.prepare_visualisation(example)
for span in doc.spans["passive"] :
    span_text = doc[span.start-2: span.end+2].text
    test_label.append([span.start_char, span.end_char, span.label_, span.start, span.end,span_text])

# removing the doublon caused by the diffferent rules
df_test = pd.DataFrame(test_label, columns = ['char_start',"char_end","label","start","end","text"])

In [108]:
df_test.sort_values("start")

,char_start,char_end,label,start,end,text
16,4003,4021,passif_tronqué,928,931,a fini par être emportées par un
17,7120,7135,passif_tronqué,1608,1611,fosse avant d' être évacués . Et
57,7323,7335,passif_factif,1650,1653,on nous a fait faire la chenille
68,7924,7940,passif_adjectif,1784,1785,encore plus incompréhensible ! Cinq
18,8166,8175,passif_tronqué,1833,1836,Donc on a été mis dans un
...,...,...,...,...,...,...
73,52419,52428,passif_adjectif,11717,11718,"preuves hyper tangibles , ça"
15,53594,53615,passif_tronqué,11977,11981,"déjà , ont été très affectés par ce"
49,54624,54640,passif_tronqué,12200,12203,parce qu' elle est ravivée régulièrement !
50,55031,55050,passif_tronqué,12287,12290,jeunes Français qui étaient touchés et qui


In [109]:
df_true

,char_start,char_end,label,start,end,text
0,3763,3803,passif,935,942,être emportées par un mouvement de foule
1,4963,4986,passif,1216,1220,deux blessés par balles
2,6735,6747,passif,1614,1616,être évacués
3,7496,7512,passif,1791,1792,incompréhensible
4,7728,7752,passif,1843,1850,on a été mis dans un bar
5,8928,8940,passif,2102,2106,Il a été tué
6,9314,9346,passif,2191,2198,on a été embarqués par la police
7,9468,9483,passif,2225,2227,être transférés
8,9485,9497,passif,2228,2232,On a été mis
9,10341,10363,passif,2428,2433,on a tous été entendus


In [24]:
t = pd.DataFrame(test_label, columns = ['start',"end","label","text"])
all_start = t['start'].tolist()



In [30]:
len(t[t['min_ecart']<30])

36

In [66]:
for l in label : 
    s = l[0]
    e = l[1]
    L = l[2]
    print(example[s:e], L)

être emportées par un mouvement de foule Passif
deux blessés par balles Passif
'être évacués Passif
incompréhensible Passif
on a été mis dans un bar Passif
Il a été tué Passif
on a été embarqués par la police Passif
être transférés Passif
On a été mis Passif
on a tous été entendus Passif
sortable Passif
être fait marcher dessus Passif
elle était recouverte d'amiante Passif
est aussi interviewé  Passif
on a été complètement assistés Passif
on est emmené Passif
on se fait piétiner Passif
je me suis fait piétiner Passif
emportée par la foule  Passif
projetée contre la scène Passif
coincé  Passif
coincé  Passif
subi Passif
intenable Passif
la rue était pas sécurisée Passif
être emmené à l'hôpital Passif
on a été évacué Passif
ils étaient aussi évacués Passif
ils ont été d'abord évacués Passif
être constamment entouré Passif
j'ai été arrêtée Passif
'il a été lourdement impacté Passif
on est toujours surveillé Passif
on est un peu couvé Passif
est complètement chamboulé Passif
'il a été vrai

In [20]:
i = 1
a = annotation['label'][0][i][0]-2
print(a)
b = annotation['label'][0][i][1]
print(b)
example[a:b]

5862
5891


"oup, ben... j'ai tout le mond"

In [23]:
annotation['label'][0]

[[4445, 4491, 'Passif'],
 [5864, 5891, 'Passif'],
 [7951, 7965, 'Passif'],
 [8847, 8863, 'Passif'],
 [9113, 9143, 'Passif'],
 [10519, 10534, 'Passif'],
 [10975, 11013, 'Passif'],
 [11156, 11172, 'Passif'],
 [11175, 11190, 'Passif'],
 [12176, 12202, 'Passif'],
 [13174, 13182, 'Passif'],
 [16315, 16342, 'Passif'],
 [16384, 16417, 'Passif'],
 [17826, 17850, 'Passif'],
 [18854, 18888, 'Passif'],
 [23862, 23877, 'Passif'],
 [23967, 23989, 'Passif'],
 [24085, 24113, 'Passif'],
 [25064, 25089, 'Passif'],
 [25230, 25257, 'Passif'],
 [25918, 25926, 'Passif'],
 [25946, 25954, 'Passif'],
 [29294, 29298, 'Passif'],
 [30207, 30216, 'Passif'],
 [30823, 30853, 'Passif'],
 [30958, 30984, 'Passif'],
 [31026, 31044, 'Passif'],
 [32000, 32028, 'Passif'],
 [32177, 32209, 'Passif'],
 [33015, 33041, 'Passif'],
 [37979, 37997, 'Passif'],
 [39289, 39321, 'Passif'],
 [40513, 40541, 'Passif'],
 [40946, 40969, 'Passif'],
 [41871, 41899, 'Passif'],
 [41942, 41973, 'Passif'],
 [45274, 45292, 'Passif'],
 [45597, 45

In [93]:
samples = analyzer.print_matches(example)


est tous complètement connectés
rule:  passif_tronqué
été complètement assistés
rule:  passif_tronqué
était très avancé
rule:  passif_tronqué
était pas sécurisée
rule:  passif_tronqué
étaient aussi évacués
rule:  passif_tronqué
être constamment entouré
rule:  passif_tronqué
sont complètement traumatisés
rule:  passif_tronqué
été lourdement impacté
rule:  passif_tronqué
est toujours surveillé
rule:  passif_tronqué
est complètement chamboulé
rule:  passif_tronqué
été vraiment perturbé
rule:  passif_tronqué
est quasiment terminé
rule:  passif_tronqué
été très affectés
rule:  passif_tronqué
suis retournée
rule:  passif_tronqué
suis couchée
rule:  passif_tronqué
être emportées
rule:  passif_tronqué
suis cachée
rule:  passif_tronqué
suis sortie
rule:  passif_tronqué
était sortie
rule:  passif_tronqué
être évacués
rule:  passif_tronqué
été mis
rule:  passif_tronqué
est sortie
rule:  passif_tronqué
été tué
rule:  passif_tronqué
suis sortie
rule:  passif_tronqué
été embarqués
rule:  passif_tron

In [11]:
test_label = []
matches, doc = analyzer.prepare_visualisation(example)
for span in doc.spans["passive"] :
    test_label.append([span.start_char, span.end_char, span.label_, span.text])

In [12]:
test_label

[[15479, 15510, 'passif_tronqué', 'est tous complètement connectés'],
 [16061, 16086, 'passif_tronqué', 'été complètement assistés'],
 [20839, 20856, 'passif_tronqué', 'était très avancé'],
 [26234, 26253, 'passif_tronqué', 'était pas sécurisée'],
 [27235, 27256, 'passif_tronqué', 'étaient aussi évacués'],
 [28094, 28118, 'passif_tronqué', 'être constamment entouré'],
 [28922, 28951, 'passif_tronqué', 'sont complètement traumatisés'],
 [33444, 33466, 'passif_tronqué', 'été lourdement impacté'],
 [34487, 34509, 'passif_tronqué', 'est toujours surveillé'],
 [35637, 35663, 'passif_tronqué', 'est complètement chamboulé'],
 [35706, 35727, 'passif_tronqué', 'été vraiment perturbé'],
 [37737, 37758, 'passif_tronqué', 'est quasiment terminé'],
 [50731, 50748, 'passif_tronqué', 'été très affectés'],
 [2017, 2031, 'passif_tronqué', 'suis retournée'],
 [3213, 3225, 'passif_tronqué', 'suis couchée'],
 [3763, 3777, 'passif_tronqué', 'être emportées'],
 [3974, 3985, 'passif_tronqué', 'suis cachée'],

In [35]:
with open('/home/robin/Code_repo/app_passive_detection/data/intransif_verbs.txt') as f : 
   lines =  f.readlines()

In [41]:
l = [elt.replace("\n","").lower().strip() for elt in  open('/home/robin/Code_repo/app_passive_detection/data/intransif_verbs.txt').readlines()[1:]]

In [43]:
len(l)

2299

In [15]:
import spacy_stanza
nlp_stanza = spacy_stanza.load_pipeline("fr", disable = ["mwt"], verbose =False)
analyzer = PassivePyAnalyzer(nlp = nlp_stanza)

example = test['clean_text'][0]
test_label = []
# compute the passive with automated method
matches, doc = analyzer.prepare_visualisation(example)
for span in doc.spans["passive"] :
    span_text = doc[span.start-2: span.end+2].text
    test_label.append([span.start_char, span.end_char, span.label_, span.start, span.end,span_text])

# removing the doublon caused by the diffferent rules
df_test = pd.DataFrame(test_label, columns = ['start',"end","label","s","e","text"])

In [ ]:
example = "je suis un des exemple du texte."
doc = nlp.make_doc(example)

In [116]:
example = "je suis un des exemple du texte."
#example = test['clean_text'][0]
doc = nlp_stanza(example)
for elt in doc.iter_tokens() :
    a = elt.start_char
    b =  elt.end_char
    print(example[a:b])
    print(elt.text)




je
je
suis
suis
un
un
des
des
exemple
exemple
du
du
texte
texte
.
.


In [ ]:

e = example.replace("\n","")
doc = nlp(e)

In [81]:
span = doc[100:120]
a = span.start_char
b =  span.end_char

print(span.text, span.start_char, span.end_char)
print(e[a:b])

de ... 17 heures pour me rendre à le concert de les Eagles of death metal , à le Bataclan 418 507
u concert des Eagles of death metal, au Bataclan, pour rejoindre mon compagnon, ses deux 


In [55]:
analyzer = PassivePyAnalyzer(nlp = nlp)


In [56]:
example = "J'ai été fatigué par la pluie. j'ai été mangé"
example = test['clean_text'][0]
test_label = []
# compute the passive with automated method
matches, doc = analyzer.prepare_visualisation(example)
for span in doc.spans["passive"] :
    span_text = doc[span.start-2: span.end+2].text
    test_label.append([span.start_char, span.end_char, span.label_, span.start, span.end,span_text])

# removing the doublon caused by the diffferent rules
df_test = pd.DataFrame(test_label, columns = ['start',"end","label","s","e","text"])

In [57]:
df_test.sort_values("start")

,start,end,label,s,e,text
50,3759,3781,passif_canonique,934,938,a fini par être emportées par un mouvement
16,3759,3777,passif_tronqué,934,937,a fini par être emportées par un
52,3979,3990,passif_impersonel,985,987,me suis cachée dans un local
17,6733,6747,passif_tronqué,1613,1616,fosse avant d'être évacués. Et
70,6927,6939,passif_factif,1655,1658,on nous a fait faire la chenille
...,...,...,...,...,...,...
49,50727,50752,passif_canonique,12188,12193,"déjà, ont été très affectés par ce qui"
63,50740,50755,passif_impersonel,12191,12194,été très affectés par ce qui s'
46,51707,51723,passif_tronqué,12411,12414,parce qu'elle est ravivée régulièrement
47,52096,52115,passif_tronqué,12498,12501,jeunes Français qui étaient touchés et qui


In [58]:
doc[934:938]

par être emportées par

In [61]:
example[3750:3781]

'n a fini par être emportées par'

In [39]:
label = test['clean_label'][0]

In [42]:
df_test['FP_found'] = df_test.apply(lambda x: check_if_found_2(x.start,x.end,label,30),axis=1)
FP_errors = df_test[df_test['FP_found']==0]
FP_errors.sort_values("start")

,start,end,label,text,FP_found
16,4003,4021,passif_tronqué,a fini par être emportées par un,0
17,7120,7135,passif_tronqué,fosse avant d' être évacués . Et,0
57,7323,7335,passif_factif,on nous a fait faire la chenille,0
68,7924,7940,passif_adjectif,encore plus incompréhensible ! Cinq,0
18,8166,8175,passif_tronqué,Donc on a été mis dans un,0
...,...,...,...,...,...
48,52137,52154,passif_tronqué,en France sans être arrêtés . Que,0
15,53594,53615,passif_tronqué,"déjà , ont été très affectés par ce",0
49,54624,54640,passif_tronqué,parce qu' elle est ravivée régulièrement !,0
50,55031,55050,passif_tronqué,jeunes Français qui étaient touchés et qui,0


In [37]:
df_true = pd.DataFrame(label, columns=["start","end","label"])
df_true['ex'] = df_true.apply(lambda x : example[x.start-2:x.end], axis=1)
df_true

,start,end,label,ex
0,3763,3803,Passif,r être emportées par un mouvement de foule
1,4963,4986,Passif,t deux blessés par balles
2,6734,6747,Passif,d'être évacués
3,7496,7512,Passif,s incompréhensible
4,7728,7752,Passif,c on a été mis dans un bar
5,8928,8940,Passif,. Il a été tué
6,9314,9346,Passif,s on a été embarqués par la police
7,9468,9483,Passif,r être transférés
8,9485,9497,Passif,. On a été mis
9,10341,10363,Passif,d on a tous été entendus


In [ ]:
 FP_errors = df_test[df_test['FP_found']==0]
    # Counting the matching passive found (labelisation is not precise) 
    df_eval = pd.DataFrame()
    df_eval['true'] = label
    df_eval['ex'] = df_eval["true"].apply(lambda x : example[x[0]-2:x[1]])
    df_eval['is_found']  = df_eval['true'].apply(lambda x : check_if_found(x,test_label, seuil))

    n_VP = len(df_eval[df_eval['is_found']==1])
    FN_errors = df_eval[df_eval['is_found']==0]
    n_true = len(df_eval['is_found'])

In [33]:
doc  = nlp("Spiderman a été vu au cinéma. La pomme a été posée sur la table ! J'ai fais à manger des pattes à mes amis.")
totalVerbs = [token for token in doc if (token.pos_ == "VERB")]
totalVerbs

/home/robin/env/app_passive/lib/python3.8/site-packages/spacy/language.py:1014: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/home/robin/env/app_passive/lib/python3.8/site-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Spiderman', 'a', 'été', 'vu', 'à', 'le', 'cinéma', '.', 'La', 'pomme', 'a', 'été', 'posée', 'sur', 'la', 'table', '!', "J'", 'ai', 'fais', 'à', 'manger', 'des', 'pattes', 'à', 'mes', 'amis', '.']
Entities: [('Spiderman', 'MISC', 0, 9), ('La pomme', 'MISC', 30, 38)]
  doc = self._ensure_doc(text)


[vu, posée, fais, manger]

In [34]:

for verb in totalVerbs:
    # direct object check
    directObject = False
    # indirect object check
    indirectObject = False
    
    
    print(verb)
    # looks through each verb's child ie dependents
    for item in verb.children:
    
        print(item.dep_)
        # sets indirectObject to true if indirect object is found
        if(item.dep_ == "iobj" or item.dep_ == "pobj"):
            indirectObject = True
        # sets directObject to true if direct object is found
        if (item.dep_ == "dobj" or item.dep_ == "dative"):
            directObject = True
            

vu
nsubj:pass
aux:tense
aux:pass
obl:arg
punct
posée
nsubj:pass
aux:tense
aux:pass
obl:mod
punct
fais
nsubj
aux:tense
xcomp
punct
manger
mark
obj
obl:mod


In [7]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_lg")
matcher = Matcher(nlp.vocab)
r = [
        {"TAG":"ADJ", "TEXT" : {"REGEX": "\b(\w*(ible|able))\b"}}, # adjectif se finnissant par ible ou able
    ]
matcher.add("test", [r], greedy='LONGEST')

In [18]:
matcher = Matcher(nlp.vocab)
r = [
        {"TAG":"ADJ","TEXT" : {"REGEX": r"\b(\w*(ible|able))\b"}}, # adjectif se finnissant par ible ou able
    ]
matcher.add("test", [r], greedy='LONGEST')

doc = nlp("cette ile est visible depuis l'ocean")
for match in matcher(doc):
    print(match)

(1618900948208871284, 3, 4)


In [8]:
data = pd.DataFrame()
data['text'] =  ["elle a été déterminée et formée ! ",
"accusé par la police, il fut condamné a 5 ans de prison ferme"
,"ma mère a été élevée par une euh une une vieille dame euh à euh dans l'Allier"
,"puis on a été soutenu par un monsieur aussi c'est monsieur NPERS…",
"la la République du Centre a été rachetée euh par un journal euh je voudrais pas dire de bêtises je crois que c'est Clermont-Ferrand Clermont enfin…",
"alors il y a des écoles qui sont faites exclusivement par des religieuses ou certains par des prêtres mais alors maintenant ils ont beaucoup d'institutrices euh civiles qui sont dedans",
"donc il est il est payé quoi bah c'est super",
"à à mon avis ça a déjà été fait mais au niveau euh professionnel au niveau du travail je crois que ça se développe ouais",
"quand tu habites dans le centre d'Orléans euh c'est quand même réservé à un public qui a … un minimum d'argent",
"parce que même avant on a été coincé euh pendant pas mal d'années euh sans voiture et on travaillait pas","tu es à peine payé et tout quoi",
"tout à l'heure j'avais été inter- la dernière fois que j'avais été interviewée c'était pour la la construction de la nouvelle fac",
"donc vous êtes  envoyée ponctuellement sur des sur des sur des sites sites",
"d'accord je devais pas rester à Orléans je devais juste être formée pendant un an et partir sur Lille",
"je trouve que pour le moment euh c'est pas encore trop bien aménagé euh pour les vélos hein vélos",
"Les adultes quand tu habites dans le centre d'Orléans euh c'est quand même réservé à un public qui a hm ouais ouais bien sûr ouais un minimum d'argent"
"des  des échanges qui qui se font pas par téléphone"]

In [3]:
data, cols = analysis_passive.load_passive_features(data, analyzer)

Detecting Sentences...
Starting to find passives...


In [2]:
text = "J'ai été attaqué par des loups !"
samples = analyzer.print_matches(text)
doc = analyzer.prepare_visualisation(text)



été attaqué
rule:  passif_tronqué
été attaqué par
rule:  passif_canonique
attaqué par des loups
rule:  passif_impersonel


In [1]:
from spacy_streamlit import visualize_spans
matches, doc = analyzer.prepare_visualisation(text)
visualize_spans(doc, spans_key="passive", show_table =  False, title = "")

NameError: name 'analyzer' is not defined

In [9]:
import spacy
nlp = spacy.load("fr_core_news_lg")
doc = nlp("je suis un exemple de text")

In [48]:
doc = nlp("elle a été déterminée et formée ! La pomme a été mangé par le chat")
ids = [(elt[1],elt[2]) for elt in passivepy.matcher(doc)]
samples = []

color_dict = {
    "passif_tronqué": "#8ef",
    "passive_rule_2" : "#faa",
    "passif_sequencé" : "afa",
    "passif_impersonel":"#fea",
    "passive_rule_5":"#faa",
    "passif_verbale":"#000080",
    "passif_factif" : "#00FF00",
    "passif_adjectif":"#808080"   
}

for token in doc :
    for id_, s,e  in passivepy.matcher(doc) :
        if not s<token.i<e :
            samples.append((token.text))
        else :
            rule_name = "test"#nlp.vocab.strings[id_]
            rule_color = "C"#color_dict[rule_name]
            samples.append((token.text,rule_name, rule_color))


In [50]:
doc.text

'elle a été déterminée et formée ! La pomme a été mangé par le chat'